---
layout: post  
title:  Analyzing Soil Microbiomes  
date: 2020-03-12  
author: Cameron Prybol  

---

```
SRR6476469
SRR6476619
SRR6476620
SRR6476646
SRR6476659
SRR6476660
SRR6476661
SRR6478609
SRR6478672
SRR6478735
SRR6478736
SRR6478752
```

In [5]:
BASE="$(homedir())/projects/2020-03-12-soil-microbiome"
if !isdir(BASE)
    mkdir(BASE)
end

"/home/jupyter-cjprybol/projects/2020-03-12-soil-microbiome"